In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Evaluate and Optimize Prompt Template Design for Better Results | Rapid Evaluation SDK Tutorial 

 <table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/prompt_engineering_evaluation_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fprompt_engineering_evaluation_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/prompt_engineering_evaluation_rapid_evaluation_sdk.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/prompt_engineering_evaluation_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Jason Dai](https://github.com/jsondai) |

## Overview

In this tutorial, you will learn how to use the use the Vertex AI Python SDK for Rapid Evaluation for Prompt Engineering & Evaluation. 


*  Use `PromptTemplate` class to automate prompt design and templating.  

*  Create an `EvalTask` using a dataset and a set of metrics.

*  Evaluate and refine the prompt templates systematically for the `EvalTask`. 

*  Compare and choose the prompt template with the best evaluation performance.

## Getting Started

### Install Vertex AI SDK for Rapid Evaluation

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[rapid_evaluation]==1.47
%pip install --quiet --upgrade nest_asyncio

### Authenticate your notebook environment (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [4]:
# General
import inspect
from uuid import uuid4
from google.colab import auth
from IPython.display import display, Markdown, HTML
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
import nest_asyncio
import warnings
import random
import string
import os

# Main
import vertexai
from vertexai.preview.evaluation import (
    EvalTask,
    PromptTemplate,
    CustomMetric,
    make_metric,
)
import pandas as pd
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, HarmCategory, HarmBlockThreshold

### Library settings

In [5]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
nest_asyncio.apply()
warnings.filterwarnings("ignore")

### Helper functions

In [7]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specifed length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown(f"### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["instruction", "context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))


def plot_radar_plot(eval_results, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )

    fig.show()


def plot_bar_plot(eval_results, metrics=None):
    fig = go.Figure()
    data = []

    for eval_result in eval_results:
        title, summary_metrics, _ = eval_result
        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()


def print_aggregated_metrics(job):
    """Print AutoMetrics"""

    rougeLSum = round(job.rougeLSum, 3) * 100
    display(
        HTML(
            f"<h3>The {rougeLSum}% of the reference summary is represented by LLM when considering the longest common subsequence (LCS) of words.</h3>"
        )
    )


def print_autosxs_judgments(df, n=3):
    """Print AutoSxS judgments in the notebook"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)

    for index, row in df.iterrows():
        if row["confidence"] >= 0.5:
            display(
                HTML(
                    f"<h2>Document:</h2> <div style='{style}'>{row['id_columns']['document']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response A:</h2> <div style='{style}'>{row['response_a']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response B:</h2> <div style='{style}'>{row['response_b']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Explanation:</h2> <div style='{style}'>{row['explanation']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Confidence score:</h2> <div style='{style}'>{row['confidence']}</div>"
                )
            )
            display(HTML("<hr>"))


def print_autosxs_win_metrics(scores):
    """Print AutoSxS aggregated metrics"""

    score_b = round(scores["autosxs_model_b_win_rate"] * 100)
    display(
        HTML(
            f"<h3>AutoSxS Autorater prefers {score_b}% of time Model B over Model A </h3>"
        )
    )

## Prompt Design and Management


Use the `PromptTemplate` class in Vertex AI SDK for Prompt Design and Prompt Management.
  
* Placeholders can be specified by `{variable}` wrapped in single curly braces, similar to the Python f-string syntax. The placeholders are compile and rendered into the text prompt when `assemble()` method is called.



### Documentation


In [8]:
print_doc(PromptTemplate)

PromptTemplate:
A prompt template for creating prompts with placeholders.

The `PromptTemplate` class allows users to define a template string with
placeholders represented in curly braces `{placeholder}`. The placeholder
names cannot contain spaces. These placeholders can be replaced with specific
values using the `assemble` method, providing flexibility in generating
dynamic prompts.

Example Usage:

    ```
        template_str = "Hello, {name}! Today is {day}. How are you?"
        prompt_template = PromptTemplate(template_str)
        completed_prompt = prompt_template.assemble(name="John", day="Monday")
        print(completed_prompt)
    ```

Attributes:
    template: The template string containing placeholders for replacement.
    placeholders: A set of placeholder names from the template string.



### Design a prompt with Prompt Template

In [9]:
prompt_template = PromptTemplate(
    "{system_instruction} Answer this question:{question}, and follow the requirements: {requirements}."
)


compiled_prompt = prompt_template.assemble(
    system_instruction="You are a poetic assistant, skilled in explaining complex concepts with creative flair.",
    question="How does LLM work?",
    requirements="Explain concepts in great depth using simple terms, and give examples to help people learn. At the end of each explanation, you ask a question to check for understanding",
)

model_response = (
    GenerativeModel("gemini-pro")
    .generate_content(str(compiled_prompt))
    .candidates[0]
    .content.parts[0]
    .text
)


display(HTML(f"<h2>Compiled Prompt:</h2><hr><h3>{compiled_prompt}</h3>"))
display(HTML(f"<h2>Model Response: </h2><hr>"))
Markdown(model_response)

**Large Language Models (LLMs): Unlocking the Secrets of AI's Linguistic Mastery**

In the vast realm of artificial intelligence, where machines strive to comprehend and generate human language, Large Language Models (LLMs) emerge as towering giants. These advanced models possess an extraordinary ability to process and interpret vast amounts of text, unlocking the secrets of language with remarkable proficiency.

**How LLMs Work: A Journey into Linguistic Labyrinth**

Imagine an LLM as a colossal tapestry, woven together by billions of tiny threads. Each thread represents a word, phrase, or grammatical rule, and the intricate pattern they form is the knowledge that the LLM has acquired through its extensive training on massive text datasets.

When you feed the LLM a piece of text, it embarks on a meticulous analysis. It breaks down the input into individual words and phrases, much like a scholar deciphering an ancient scroll. The LLM then examines the relationships between these linguistic elements, identifying patterns and connections that govern the structure and meaning of the text.

By leveraging its vast knowledge of language, the LLM can perform a wide range of tasks:

* **Text generation:** LLMs can effortlessly create new text that mimics human writing, from captivating stories to informative articles. They do this by predicting the next word in a sequence based on the preceding words, just as a writer would.
* **Language translation:** LLMs can seamlessly translate text from one language to another, bridging the gap between different cultures and fostering communication.
* **Question answering:** LLMs possess the ability to extract information from text and provide concise, human-readable answers to questions.
* **Summarization:** LLMs can condense large chunks of text into concise summaries, capturing the key points and delivering them in a clear and succinct manner.

**Examples to Illuminate the LLM's Brilliance**

* **A Chatbot Powered by an LLM:** Imagine a chatbot that can engage in natural language conversations, responding to your questions with coherence and wit. This is made possible by an underlying LLM that understands the context of your inquiries and generates appropriate replies.
* **An AI-Generated Poem:** An LLM can weave words together to create original poems that mimic the style and themes of renowned poets. These creations demonstrate the LLM's ability to comprehend the nuances of language and produce creative content.

**Question for Understanding:**

In your own words, explain how LLMs analyze and process text.

##  Compare and optimize prompt template design for Prompt Engineering

The Vertex AI SDK streamlines the structured compilation of text prompts from datasets and provides comprehensive evaluation results of multiple Prompt Templates for a specified evaluation task that can be compared for prompt engineering.

### Define an evaluation dataset

In [16]:
instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]


eval_dataset = pd.DataFrame(
    {
        "context": context,
        "instruction": [instruction] * len(context),
        "reference": reference,
    }
)

### Define prompt templates to compare



In [17]:
prompt_templates = [
    "Instruction: {instruction}. Article: {context}. Summary:",
    "Article: {context}. Complete this task: {instruction}, in one sentence. Summary:",
    "Goal: {instruction} and give me a TLDR. Here's an article: {context}. Summary:",
]

### Define metrics

In [18]:
metrics = [
    "rouge_1",
    "rouge_l_sum",
    "bleu",
    "fluency",
    "coherence",
    "safety",
    "groundedness",
    "summarization_quality",
    "summarization_verbosity",
    "summarization_helpfulness",
]

### Define EvalTask & Experiment

In [19]:
generation_config = {
    "temperature": 0.3,
}

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

gemini_model = GenerativeModel(
    "gemini-pro", generation_config=generation_config, safety_settings=safety_settings
)

In [20]:
experiment_name = "eval-sdk-prompt-engineering"  # @param {type:"string"}

summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=experiment_name,
)

### Running Evaluation

In [21]:
run_id = generate_uuid()
eval_results = []


for i, prompt_template in tqdm(
    enumerate(prompt_templates), total=len(prompt_templates)
):
    experiment_run_name = f"eval-prompt-engineering-{run_id}-prompt-{i}"

    eval_result = summarization_eval_task.evaluate(
        prompt_template=prompt_template,
        experiment_run_name=experiment_run_name,
        model=gemini_model,
    )

    eval_results.append(
        (f"Prompt #{i}", eval_result.summary_metrics, eval_result.metrics_table)
    )

  0%|          | 0/3 [00:00<?, ?it/s]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-sdk-prompt-engineering-eval-prompt-engineering-ltcp4v0h-prompt-0 to Experiment: eval-sdk-prompt-engineering
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': 'Instruction: {instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-pro', 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


 33%|███▎      | 1/3 [00:22<00:45, 22.81s/it]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-sdk-prompt-engineering-eval-prompt-engineering-ltcp4v0h-prompt-1 to Experiment: eval-sdk-prompt-engineering
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': 'Article: {context}. Complete this task: {instruction}, in one sentence. Summary:', 'model_name': 'publishers/google/models/gemini-pro', 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


 67%|██████▋   | 2/3 [00:46<00:23, 23.11s/it]

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/eval-sdk-prompt-engineering-eval-prompt-engineering-ltcp4v0h-prompt-2 to Experiment: eval-sdk-prompt-engineering
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'prompt_template': "Goal: {instruction} and give me a TLDR. Here's an article: {context}. Summary:", 'model_name': 'publishers/google/models/gemini-pro', 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}


100%|██████████| 3/3 [01:19<00:00, 26.62s/it]


### Display Evaluation report and explanations

In [22]:
for eval_result in eval_results:
    display_eval_report(eval_result)

## Prompt #0

### Summary Metrics

,row_count,rouge_1/mean,rouge_1/std,rouge_l_sum/mean,rouge_l_sum/std,bleu/mean,bleu/std,fluency/mean,fluency/std,coherence/mean,...,safety/mean,safety/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,summarization_verbosity/mean,summarization_verbosity/std,summarization_helpfulness/mean,summarization_helpfulness/std
0,5.0,0.413917,0.116888,0.338255,0.103016,0.090159,0.054427,4.4,0.547723,2.4,...,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0


### Report Metrics

,context,instruction,reference,completed_prompt,response,rouge_1,rouge_l_sum,bleu,fluency/explanation,fluency/confidence,...,groundedness,summarization_quality/explanation,summarization_quality/confidence,summarization_quality,summarization_verbosity/explanation,summarization_verbosity/confidence,summarization_verbosity,summarization_helpfulness/explanation,summarization_helpfulness/confidence,summarization_helpfulness
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Instruction: Summarize the following article. ...,"To prepare spaghetti carbonara, boil salted wa...",0.465753,0.356164,0.128228,The response provides a clear and concise set ...,0.6,...,1.0,The response follows the instruction to summar...,1.0,5.0,The response provides all the necessary steps ...,1.0,0.0,The response provides all the important detail...,1.0,5.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Instruction: Summarize the following article. ...,Risotto preparation involves sautéing onions a...,0.557377,0.491803,0.166569,The response has no grammatical errors and use...,0.9,...,1.0,The summary in response follows instructions t...,0.8,5.0,The response is concise and provides all the n...,1.0,0.0,The response captures all the key steps in pre...,1.0,5.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Instruction: Summarize the following article. ...,"To grill a flavorful steak, select a well-marb...",0.342857,0.285714,0.055412,The response demonstrates correct use of gramm...,0.9,...,1.0,The summary in response follows instructions t...,1.0,5.0,The response provides concise instructions for...,1.0,0.0,The response captures all the important detail...,0.9,5.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Instruction: Summarize the following article. ...,"To make creamy tomato soup, sauté onions and g...",0.448276,0.344828,0.059651,The response provides clear and concise instru...,0.8,...,1.0,The response follows the instruction to summar...,0.9,5.0,The response provides all the necessary steps ...,1.0,0.0,The response captures all the important steps ...,0.8,5.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Instruction: Summarize the following article. ...,"To make a chocolate cake from scratch, preheat...",0.255319,0.212766,0.040933,"The response has very few, if any, minor gramm...",0.8,...,1.0,The summary in response follows the instructio...,1.0,5.0,The response provides all the necessary steps ...,1.0,0.0,The summary captures all the important steps a...,1.0,5.0


...

## Prompt #2

### Summary Metrics

,row_count,rouge_1/mean,rouge_1/std,rouge_l_sum/mean,rouge_l_sum/std,bleu/mean,bleu/std,fluency/mean,fluency/std,coherence/mean,...,safety/mean,safety/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,summarization_verbosity/mean,summarization_verbosity/std,summarization_helpfulness/mean,summarization_helpfulness/std
0,5.0,0.33281,0.094272,0.283721,0.086654,0.0537,0.029377,4.6,0.547723,2.0,...,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0


### Report Metrics

,context,instruction,reference,completed_prompt,response,rouge_1,rouge_l_sum,bleu,fluency/explanation,fluency/confidence,...,groundedness,summarization_quality/explanation,summarization_quality/confidence,summarization_quality,summarization_verbosity/explanation,summarization_verbosity/confidence,summarization_verbosity,summarization_helpfulness/explanation,summarization_helpfulness/confidence,summarization_helpfulness
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Goal: Summarize the following article and give...,**Summary:**\n\nTo prepare spaghetti carbonara...,0.333333,0.270833,0.079797,The response is well-written and easy to under...,0.9,...,1.0,The summary in response follows the instructio...,1.0,5.0,The summary is concise and provides all the ne...,1.0,0.0,The response captures all the important detail...,1.0,5.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Goal: Summarize the following article and give...,"**Summary:**\n\nTo make a perfect risotto, fol...",0.404040,0.343434,0.036649,The response has no grammatical errors and use...,0.5,...,1.0,The summary in the response follows instructio...,0.9,5.0,The summary is concise and provides all the ne...,1.0,0.0,The summary captures all the important steps i...,1.0,5.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Goal: Summarize the following article and give...,"**Summary:**\n\nTo grill a flavorful steak, se...",0.275862,0.229885,0.035204,The response is free of grammatical errors and...,1.0,...,1.0,The summary in response follows the instructio...,0.8,5.0,The summary is concise and provides all the ne...,1.0,0.0,The response captures all the important detail...,0.9,5.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Goal: Summarize the following article and give...,**Summary:**\n\nCreating creamy homemade tomat...,0.441860,0.395349,0.090778,The response is well-written and easy to under...,0.9,...,1.0,The summary in response follows the instructio...,0.9,5.0,The summary is concise and provides all the ne...,1.0,0.0,The summary captures all the important steps i...,1.0,5.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Goal: Summarize the following article and give...,**Summary:**\n\nTo bake a chocolate cake from ...,0.208955,0.179104,0.026071,The response is written in a clear and concise...,0.8,...,1.0,The summary in response follows the instructio...,1.0,5.0,The summary is concise and provides all the ne...,1.0,0.0,The response provides all the important detail...,1.0,5.0


In [23]:
for eval_result in eval_results:
    display_explanations(eval_result[2], metrics=["summarization_verbosity"])

### Compare Eval Results

In [24]:
plot_radar_plot(
    eval_results,
    metrics=[
        f"{metric}/mean"
        for metric in [
            "fluency",
            "coherence",
            "summarization_quality",
            "summarization_helpfulness",
        ]
    ],
)
plot_bar_plot(
    eval_results,
    metrics=[f"{metric}/mean" for metric in ["rouge_1", "rouge_l_sum", "bleu"]],
)

In [25]:
summarization_eval_task.display_runs()

,experiment_name,run_name,run_type,state,param.model_name,param.HARM_CATEGORY_DANGEROUS_CONTENT,param.prompt_template,param.HARM_CATEGORY_HARASSMENT,param.HARM_CATEGORY_UNSPECIFIED,param.HARM_CATEGORY_HATE_SPEECH,...,metric.row_count,metric.bleu/std,metric.coherence/std,metric.safety/mean,metric.bleu/mean,metric.summarization_helpfulness/std,metric.rouge_1/mean,metric.rouge_l_sum/std,metric.fluency/std,metric.summarization_verbosity/std
0,eval-sdk-prompt-engineering,eval-prompt-engineering-ltcp4v0h-prompt-2,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Goal: {instruction} and give me a TLDR. Here's...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.029377,0.000000,1.0,0.053700,0.000000,0.332810,0.086654,0.547723,0.000000
1,eval-sdk-prompt-engineering,eval-prompt-engineering-ltcp4v0h-prompt-1,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Article: {context}. Complete this task: {instr...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.113353,0.894427,1.0,0.204766,0.894427,0.554697,0.127181,0.447214,0.000000
2,eval-sdk-prompt-engineering,eval-prompt-engineering-ltcp4v0h-prompt-0,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Instruction: {instruction}. Article: {context}...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.054427,0.894427,1.0,0.090159,0.000000,0.413917,0.103016,0.547723,0.000000
3,eval-sdk-prompt-engineering,eval-prompt-engineering-ja78lvl6-prompt-2,system.ExperimentRun,FAILED,publishers/google/models/gemini-pro,BLOCK_NONE,Goal: {instruction} and give me a TLDR. Here's...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,eval-sdk-prompt-engineering,eval-prompt-engineering-ja78lvl6-prompt-1,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Article: {context}. Complete this task: {instr...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.099306,0.000000,1.0,0.176024,1.095445,0.555298,0.126247,1.000000,0.000000
5,eval-sdk-prompt-engineering,eval-prompt-engineering-ja78lvl6-prompt-0,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Instruction: {instruction}. Article: {context}...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.011706,1.095445,1.0,0.047491,0.447214,0.359930,0.076151,0.547723,0.000000
6,eval-sdk-prompt-engineering,eval-prompt-engineering-4sgcz42s-prompt-2,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Goal: {instruction} and give me a TLDR. Here's...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.036096,0.000000,1.0,0.077684,0.894427,0.370749,0.095432,1.414214,0.000000
7,eval-sdk-prompt-engineering,eval-prompt-engineering-4sgcz42s-prompt-1,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Article: {context}. Complete this task: {instr...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.122641,0.000000,1.0,0.197136,1.095445,0.529465,0.145343,1.224745,0.000000
8,eval-sdk-prompt-engineering,eval-prompt-engineering-4sgcz42s-prompt-0,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-pro,BLOCK_NONE,Instruction: {instruction}. Article: {context}...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,5.0,0.005204,1.095445,1.0,0.052014,0.000000,0.389950,0.092557,0.836660,0.000000
9,eval-sdk-prompt-engineering,eval-prompt-engineering-0vbfi2l4-prompt-0,system.ExperimentRun,FAILED,publishers/google/models/gemini-pro,BLOCK_NONE,Instruction: {instruction}. Article: {context}...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Comment:** You can evaluate your prompts by passing metric names individually. Rapid Evaluation SDK supports metric-bundles which group metrics based on their tasks/criterias/inputs to facilitate convenient usage. For example, when you have a task in mind (e.g. summarization), you can pick the metric bundle `summarization_pointwise_reference_free` for this task directly to generate all relevant metrics.